In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, Resize, Normalize, RandomHorizontalFlip, RandomRotation, RandomAffine

In [3]:
test_files = ['']

In [4]:
# Fix the CustomDataset class to keep images as 1-channel (grayscale)
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Reshape data to (1, H, W) for single grayscale channel
        image = self.data[idx].astype(np.float32).reshape(40, 168)

        label = self.labels[idx]

        # Convert to PyTorch Tensor and apply transformations
        if self.transform:
            # image = torch.tensor(image)  # Convert to PyTorch Tensor
            image = self.transform(image)

        return image, label

In [5]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [6]:
test_data = CustomDataset(data[24000:], labels[24000:], transform=None)

NameError: name 'data' is not defined

In [7]:
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

NameError: name 'test_data' is not defined

In [8]:
# Load the model architecture
model = models.resnet18(pretrained=False)  # Load ResNet18 architecture
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Update for 1-channel input
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Update the final layer for the number of classes

# Load the saved state dictionary
model.load_state_dict(torch.load("resnet_model.pth"))

# Set the model to evaluation mode
model.eval()

/usr/local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


NameError: name 'num_classes' is not defined

In [ ]:
# Assuming `test_loader` is prepared
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Perform inference
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)
        print("Predictions:", predictions.cpu().numpy())
        print("True Labels:", labels.numpy())
        break  # Stop after the first batch


In [ ]:
model = torch.load("resnet_complete_model.pth")
model.eval()